In [1]:
from fast_to_SQL.fast_to_SQL import to_sql_fast
import pandas as pd
from urllib.parse import quote_plus
import sqlalchemy
%matplotlib inline

params = quote_plus(
    """DRIVER={ODBC Driver 13 for SQL Server};
    SERVER=server-dw01.database.windows.net;
    DATABASE=DataWarehouseStaging;
    UID=dw_userlogin;
    PWD=kljDueK!7"""
)

ENGINE = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [2]:
month_no_dict = {'jan' : 1,
'feb' : 2,
'mar' : 3,
'apr' : 4,
'may' : 5,
'jun' : 6,
'jul' : 7,
'aug' : 8,
'sep' : 9,
'oct' : 10,
'nov' : 11,
'dec' : 12
}

In [3]:
Xer_df = pd.read_sql(''' SELECT * FROM pam.dataset_Xer ''', ENGINE)

In [4]:
df = pd.melt(Xer_df, id_vars = ['com_id_h'], var_name = 'month', value_name = 'Xer')

In [5]:
df['month_no'] = df['month'].map(month_no_dict)

In [6]:
mktg_df = pd.read_sql(''' SELECT [Hashed_ID], [Land_Date], [Opportunity ID], [MCC_Description], t.[Tenure]
	,t.[Average_Turnover]
	,t.[Average_Trans_Count]
	,t.[Times_Marketed_To]
	,t.[True PA or Eligible] AS [Message]
    ,t.[Pam_Score]
                            FROM [marketing].[WP_Attri_Mailers] AS t
                            WHERE Marketed_To = 1''', ENGINE)

In [7]:
mktg_df['Land_Date'] = pd.to_datetime(mktg_df['Land_Date'])

mktg_df['land_month'] = pd.DatetimeIndex(mktg_df['Land_Date']).month

import calendar
mktg_df['land_month'] = mktg_df['land_month'].apply(lambda x: calendar.month_abbr[x])

mktg_df['land_month'] = mktg_df['land_month'].str.lower()

In [8]:
mktg_df['land_month_no'] = mktg_df['land_month'].map(month_no_dict)

In [9]:
mktg_df['responded'] = mktg_df['Opportunity ID'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [10]:
final_df = mktg_df.merge(right = df, how = 'left', left_on = ['Hashed_ID','land_month_no'], right_on = ['com_id_h', 'month_no'])

In [11]:
final_df = final_df[['Hashed_ID', 'Tenure',
       'Average_Turnover', 'Average_Trans_Count', 'Times_Marketed_To',
       'Message', 'Xer', 'MCC_Description', 'Pam_Score', 'responded']]

In [12]:
final_df.to_csv('propensity_dataset_MCCraw.csv', index = False)